# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import (weather_api_key,g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
import_csv = "../WeatherPy/city_data.csv"    
data_df = pd.read_csv(import_csv,encoding='utf-8')
data_df

,Unnamed: 0,City,Lat,Lon,Temp,Max Temp,Cloudiness,Wind Speed,Country,Date,Humidity
0,0,Codrington,-38.2667,141.9667,54.45,54.45,100,4.43,AU,1619384808,81
1,1,Saint-Philippe,-21.3585,55.7679,72.09,78.80,20,23.02,RE,1619384809,69
2,2,Tasiilaq,65.6145,-37.6368,37.40,37.40,75,19.57,GL,1619384794,81
3,3,Hermanus,-34.4187,19.2345,62.01,62.01,100,4.00,ZA,1619384809,88
4,4,Mataura,-46.1927,168.8643,46.00,46.00,66,13.00,NZ,1619384810,80
...,...,...,...,...,...,...,...,...,...,...,...
576,576,Pakhtakoron,40.1571,68.7466,63.46,63.46,5,4.94,TJ,1619385013,50
577,577,Coos Bay,43.3665,-124.2179,52.29,54.00,90,17.27,US,1619384983,81
578,578,Lavrentiya,65.5833,-171.0000,29.88,29.88,94,1.72,RU,1619385014,84
579,579,Forsytheganj,52.6333,29.7333,31.96,31.96,11,5.95,BY,1619385014,78


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
marker_l = data_df[['Lat','Lon']].astype(float) 
fig = gmaps.figure(layout={
        'width': '500px',
        'height': '400px',
        'padding': '3px',
        'border': '1px solid black'
})
heatmap_layer = gmaps.heatmap_layer(marker_l, weights=data_df['Humidity'],max_intensity=100, point_radius=20.0
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='3px', width='500px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [4]:
hotel_df = data_df.loc[(data_df['Temp']<80) & (data_df['Temp']>70) & (data_df['Wind Speed']<10) & (data_df['Cloudiness'] ==0)] 
hotel_df = hotel_df.dropna()
hotel_df['Hotel Name'] = ""
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_df

,Unnamed: 0,City,Lat,Lon,Temp,Max Temp,Cloudiness,Wind Speed,Country,Date,Humidity,Hotel Name
82,82,Beloha,-25.1667,45.0500,73.13,73.13,0,7.65,MG,1619384836,74,
136,136,Arraial do Cabo,-22.9661,-42.0278,74.89,77.00,0,6.91,BR,1619384854,88,
162,162,Balkanabat,39.5108,54.3671,76.19,76.19,0,1.81,TM,1619384863,31,
183,183,Palmerston,-12.4860,130.9833,70.48,73.40,0,3.44,AU,1619384870,73,
186,186,Vila Velha,-20.3297,-40.2925,78.80,78.80,0,5.75,BR,1619384832,83,
211,211,Karratha,-20.7377,116.8463,75.47,75.47,0,7.83,AU,1619384880,59,
319,319,São João da Barra,-21.6403,-41.0511,78.80,78.80,0,9.22,BR,1619384919,73,
325,325,Phuntsholing,26.8516,89.3884,75.70,75.70,0,8.08,BT,1619384922,43,
388,388,Xai-Xai,-25.0519,33.6442,70.48,70.48,0,3.11,MZ,1619384944,84,
397,397,Kiryat Gat,31.6100,34.7642,73.33,73.99,0,1.99,IL,1619384947,39,


In [5]:
from pprint import pprint
hoteloc = '22.2006, 113.5461'
target_search = 'hotel'
params = {'location': hoteloc,
          'keyword':target_search,
          'radius': 5000,
          'key': g_key}
hotel1 = requests.get(url,params).json()
hotel1['results'][0]['name']

'The St. Regis Zhuhai'

In [6]:
new = hotel_df.astype(str)
lat_lon = new['Lat'].str.cat(new['Lon'],sep=',')
hotel_name = []
hotel_lat = []
hotel_lon = []
for latlons in lat_lon:
    try: 
        params = {'location': latlons,
             'keyword':target_search,
             'radius':5000,
             'key':g_key}
        hotel2=requests.get(url,params).json()
        print(hotel2['results'][0]['name'])
        hotel_name.append(hotel2['results'][0]['name'])
        hotel_lat.append(hotel2['results'][0]['geometry']['location']['lat'])
        hotel_lon.append(hotel2['results'][0]['geometry']['location']['lng'])


    except: 
        print('That is too far baby')
        hotel_name.append('that is too far baby')
        hotel_lat.append('no data here')
        hotel_lon.append('no data here')
hotel_df['Hotel Name'] = hotel_name


That is too far baby
Ocean View Hotel
serdarlan oyi
Rydges Palmerston - Darwin
Hotel Senac Ilha do Boi
Karratha International Hotel
Pousada Kactus
Hotel Phuentsholing
That is too far baby
Desert Gat
Dana North Apartment
BIG4 Howard Springs Holiday Park
Four Seasons Hotel Riyadh At Kingdom Center
That is too far baby


In [7]:
hotel_df['Hotel Lat']=hotel_lat
hotel_df['Hotel Lon']=hotel_lon
final_hotel_data = hotel_df.loc[hotel_df['Hotel Name']!='that is too far baby']


In [8]:
final_hotel_data


,Unnamed: 0,City,Lat,Lon,Temp,Max Temp,Cloudiness,Wind Speed,Country,Date,Humidity,Hotel Name,Hotel Lat,Hotel Lon
136,136,Arraial do Cabo,-22.9661,-42.0278,74.89,77.00,0,6.91,BR,1619384854,88,Ocean View Hotel,-22.9588,-42.0259
162,162,Balkanabat,39.5108,54.3671,76.19,76.19,0,1.81,TM,1619384863,31,serdarlan oyi,39.5247,54.3658
183,183,Palmerston,-12.4860,130.9833,70.48,73.40,0,3.44,AU,1619384870,73,Rydges Palmerston - Darwin,-12.481,130.989
186,186,Vila Velha,-20.3297,-40.2925,78.80,78.80,0,5.75,BR,1619384832,83,Hotel Senac Ilha do Boi,-20.3106,-40.2827
211,211,Karratha,-20.7377,116.8463,75.47,75.47,0,7.83,AU,1619384880,59,Karratha International Hotel,-20.7376,116.842
319,319,São João da Barra,-21.6403,-41.0511,78.80,78.80,0,9.22,BR,1619384919,73,Pousada Kactus,-21.6654,-41.0232
325,325,Phuntsholing,26.8516,89.3884,75.70,75.70,0,8.08,BT,1619384922,43,Hotel Phuentsholing,26.8639,89.3838
397,397,Kiryat Gat,31.6100,34.7642,73.33,73.99,0,1.99,IL,1619384947,39,Desert Gat,31.6027,34.7643
439,439,Qurayyat,31.3318,37.3428,73.40,73.40,0,9.22,SA,1619384964,23,Dana North Apartment,31.3201,37.3463
445,445,Howard Springs,-12.4970,131.0470,70.11,73.40,0,3.44,AU,1619384966,73,BIG4 Howard Springs Holiday Park,-12.4973,131.034


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in final_hotel_data.iterrows()]
final_hotel_data

,Unnamed: 0,City,Lat,Lon,Temp,Max Temp,Cloudiness,Wind Speed,Country,Date,Humidity,Hotel Name,Hotel Lat,Hotel Lon
136,136,Arraial do Cabo,-22.9661,-42.0278,74.89,77.00,0,6.91,BR,1619384854,88,Ocean View Hotel,-22.9588,-42.0259
162,162,Balkanabat,39.5108,54.3671,76.19,76.19,0,1.81,TM,1619384863,31,serdarlan oyi,39.5247,54.3658
183,183,Palmerston,-12.4860,130.9833,70.48,73.40,0,3.44,AU,1619384870,73,Rydges Palmerston - Darwin,-12.481,130.989
186,186,Vila Velha,-20.3297,-40.2925,78.80,78.80,0,5.75,BR,1619384832,83,Hotel Senac Ilha do Boi,-20.3106,-40.2827
211,211,Karratha,-20.7377,116.8463,75.47,75.47,0,7.83,AU,1619384880,59,Karratha International Hotel,-20.7376,116.842
319,319,São João da Barra,-21.6403,-41.0511,78.80,78.80,0,9.22,BR,1619384919,73,Pousada Kactus,-21.6654,-41.0232
325,325,Phuntsholing,26.8516,89.3884,75.70,75.70,0,8.08,BT,1619384922,43,Hotel Phuentsholing,26.8639,89.3838
397,397,Kiryat Gat,31.6100,34.7642,73.33,73.99,0,1.99,IL,1619384947,39,Desert Gat,31.6027,34.7643
439,439,Qurayyat,31.3318,37.3428,73.40,73.40,0,9.22,SA,1619384964,23,Dana North Apartment,31.3201,37.3463
445,445,Howard Springs,-12.4970,131.0470,70.11,73.40,0,3.44,AU,1619384966,73,BIG4 Howard Springs Holiday Park,-12.4973,131.034


In [10]:
# Add marker layer ontop of heat map
marker_locs = final_hotel_data[['Hotel Lat','Hotel Lon']]
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locs,info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))